# Challenge


Our task is to forecast the total amount of products sold in every shop for the test set.
Model should handle shops and products changing each month


 ## Files

Training Set: `sales_train.csv`

Test set: `test.csv`: Forecast sales for these shops 

`sample_submission.csv`: the expected format of a submission

`items.csv`: supplemental info about items/products

`item_categories.csv`: supplemental info about item categories

`shops.csv`: supplemental info about shops

 ## Variables 

`ID`: represents a shop item within the test set

`shop_id`: unique identifier of a shop

`item_id`: unique identifier of a product

`item_category_id`: unique identifier of an item category

`item_cnt_day`: number of products sold in a day

`item_price`: current price of item

`date`: dd/mm/yyyy

`date_Block_num`: January 2013 is 0. Increment every month until October 2015 (33)

`item_name`: name of item

`shop_name`: name of shop

`item_category_name`: name of item category


We are going to ignore data such as names of items, shops and item categories, as we will not be using text analysis to look for trends in sales based on name similarity, only on the items, shops and categories themselves. As such, ids are more useful. Additionally, `DATE` will be transformed into more useful data that ignores years.


The input to the model should be the variables `ID`, `shop_id`, and `item_id`, therefore, we must train a model that can recognize monthly trends in sales per shop per item to predict next month's data from this month's. 

Our model also knows that the current `date_block_num` is 34 because we are predicting for just November 2015, so we can use `date`, and `date_block_num` as part of the model input.

# Visualization

It is necessary that we combine `item_cnt_day` into `item_cnt_month` for each item in each shop and graph to see what kind of trends each item in each shop have over the past two years. We can overlay the data for each item with everyshop that carried it that month.

In [13]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
DATA_DIR = "~/.kaggle/competitions/competitive-data-science-final-project/"

# TODO:
# read in csv file, remove headers and date, sort by date_block_num by item_id by store_id with custom sort criteria
# aggregate variable item_cnt_day in last column for each store in date_block_num as new row item_cnt_month
train_set = pd.read_csv(DATA_DIR + "sales_train.csv")  
train_set=train_set.iloc[: , 1:]
train_set = train_set.sort_values(by=["date_block_num", "shop_id","item_id"])
#train_set = train_set.agg(["sum"])
#print(train_set)


test = pd.DataFrame({"year": [0,0,0,0,1,1,1,2,2,2,2,3,3,3,3], "month" : [0,0,0,0,1,1,1,2,2,2,3,3,3,4,4], "day" : [0,0,0,1,1,1,2,2,2,2,3,3,4,4,5], "day_count" : [7,4,3,2,1,5,4,2,3,2,5,3,2,1,3]})
test = test[["year", "month", "day", "day_count"]]
#print(test)

def agg_multiple(df, labels, aggvar, repl=None):
    if(repl is None): repl = aggvar
    conds = df.duplicated(labels).tolist() #returns boolean list of false for a unique (year,month) then true until next unique pair
    groups = []
    start = 0
    for i in range(len(conds)): #When false, split previous to new df, aggregate count 
        bul = conds[i]
        if(i == len(conds) - 1): i +=1 #no false marking end of last group, special case
        if not bul and i > 0 or bul and i == len(conds): 
            sample = df.iloc[start:i , :]
            start = i
            sample = sample.groupby(labels, as_index=False).agg({aggvar:sum}).rename(columns={aggvar : repl})
            groups.append(sample)
    df = pd.concat(groups).reset_index(drop=True) #combine aggregated dfs into new df
    return df
test2 = agg_multiple(test, ["year", "month"], "day_count", repl="month_count")
print(test2)
%timeit test.groupby(["year", "month"], as_index=False).agg({"day_count":sum}).rename(columns={"day_count":"month_count"})

%timeit test.groupby(['year', 'month']).day_count.sum().to_frame('month_count').reset_index()


def plotrandom(train_set, items, stores):
    for i in range(stores):
        store = np.random.choice(train_set["shop_id"])
        for j in range(items):
            
            train_set2 = train_set.loc[train_set['shop_id'] == store]        
            item = random.choice(train_set['item_id'])
            train_set2 = train_set.loc[train_set['item_id'] == item]


            train_set2 = agg_multiple(train_set2, ["date_block_num", "shop_id", "item_id"], "item_cnt_day", repl="item_cnt_mnth")

            #TODO: Graph month by item_cnt_month for distinct item_id, and draw a seperate line for 5 chosen stores
            #####  Repeat for 5 items
            #####  Analyze graphs
            x = train_set2["date_block_num"]
            y = train_set2["item_cnt_mnth"]
            plt.plot(x,y)
        plt.show()
#plotrandom(train_set, 2,2)




   year  month  month_count
0     0      0           16
1     1      1           10
2     2      2            7
3     2      3            5
4     3      3            5
5     3      4            4
2.86 ms ± 146 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
2 ms ± 90.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [15]:
dataset = agg_multiple(train_set.sample(frac=.01), ["date_block_num", "shop_id", "item_id"], "item_cnt_day", repl="item_cnt_mnth")
dataset[dataset["item_cnt_mnth"] > 20] = 20
print(dataset)
train_set = dataset.sample(frac=.75)
test_set =  dataset.sample(frac=.25)


       date_block_num  shop_id  item_id  item_cnt_mnth
0                   2       47    12472            1.0
1                   9       25    14829            6.0
2                  11       25    11616            1.0
3                   8       27     9659            1.0
4                  24       48     6472            1.0
5                  16       57     8415            2.0
6                  18       24     6316            1.0
7                  23       25    16203            1.0
8                   0       41    11041            2.0
9                   8        6    16832            1.0
10                 23       15     3341            3.0
11                  3        3    15369            1.0
12                 23       31     1556            1.0
13                  9       55     9524            1.0
14                  8       25     7098            2.0
15                  0       31    13605            1.0
16                  0       41    19164            1.0
17        

There is no obvious pattern in the graphs for any item along stores, or stores along items. We assume the two are not independent.

# Model

For this problem we will be using polynomial regression, random forests, and an SVM, optimizing each, and then comparing the results on the validation set, which will be a small split from the training set. If all perform well, a voting system will be installed and all three will be responsible for determining each prediction for each store and item as a rudimentary ensemble method.


The file `tests.csv` contains the set of all training data. We must read that in, and create a prediction for each `ID`. that `ID` should then be marked in the new submission csv before the prediction. `ID` is redundant with `shop_id` and `item_id`, so we will not train with it. 

### Polynomial Regression
Polynomial regression isn't too difficult to model. Using the library scikit learn, we simply need to seperate the expected from the training matrix and run a few functions. We can then make predictions based on new data.

In [16]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression



X = np.array(train_set[["date_block_num", "shop_id","item_id"]])
X_test =  np.array(test_set[["date_block_num", "shop_id","item_id"]])
y = np.array(train_set[["item_cnt_mnth"]])
y_test = np.array(test_set[["item_cnt_mnth"]])
# PolynomialFeatures (prepreprocessing)
poly = PolynomialFeatures(degree=10, include_bias=True)
X_ = poly.fit_transform(X)
X_test_ = poly.fit_transform(X_test)


# Instantiate
lg = LinearRegression()

# Fit
lg.fit(X_, y)

# Obtain coefficients
lg.coef_



print(X_)
preds = np.round(lg.predict(X_test_))
print(preds)
print(lg.score(X_test_, y_test))
#accuracy function
# find absolute difference between predictions and values
# add differences together
# take 1 - number of differences/total sum


#true accuracy function
# add up total correct, divide by total
true = np.sum((preds == y_test.ravel )) / len(y_test) 
print("true accuracy is ", true)

preds = np.round(lg.predict(X_))
true = np.sum((preds == y.ravel )) / len(y) 
print("true accuracy is ", true)


[[1.00000000e+00 1.80000000e+01 2.60000000e+01 ... 3.65599322e+29
  3.08790812e+31 2.60809471e+33]
 [1.00000000e+00 2.00000000e+00 3.00000000e+01 ... 1.15564191e+34
  2.98040049e+36 7.68645287e+38]
 [1.00000000e+00 2.30000000e+01 3.00000000e+01 ... 5.57390580e+27
  2.33360856e+29 9.77004117e+30]
 ...
 [1.00000000e+00 2.50000000e+01 5.20000000e+01 ... 5.55412280e+31
  3.69562786e+33 2.45901392e+35]
 [1.00000000e+00 1.40000000e+01 4.10000000e+01 ... 1.49393133e+36
  4.78786773e+38 1.53445322e+41]
 [1.00000000e+00 1.30000000e+01 2.70000000e+01 ... 2.53304596e+31
  3.46652031e+33 4.74399724e+35]]
[[1.]
 [1.]
 [1.]
 ...
 [1.]
 [1.]
 [1.]]
-0.003803313929410024
true accuracy is  0.0
true accuracy is  0.0


This model is clearly unsuccessful, with a correlation coefficient of close to 0. It achieves a prediction accuracy of 20% through shear overfitting. The data is clearly not well-mapped by a polynomial. 

### Random Forest
To model the data as a random forest we can again use the sklearn random forest regression in the ensemble library.


In [17]:
from sklearn.ensemble import RandomForestRegressor

#np.nan_to_num(X_test, copy=0)


regr = RandomForestRegressor(n_estimators = 50, max_depth=None, random_state=37, oob_score=1)
regr.fit(X, y.ravel())
preds = np.round(regr.predict(X_test))
print(regr.score(X_test, y_test.ravel()))
#accuracy function
# find absolute difference between predictions and values
# add differences together
# take 1 - number of differences/total sum
pred = np.round(regr.predict(X))

#true accuracy function
# add up total correct, divide by total
true = np.sum((preds == y_test.ravel() )) / len(y_test) 

print("true accuracy is ", true)


0.8079589865289066
true accuracy is  0.9182561307901907


Random Forests are not good for problems where the nodes will encounter features they've never seen, that is why the test set only has 37% success, no matter how we try to retune the parameters. This dataset requires a more flexible model.

### SVM

In [20]:
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV


#postval_set = agg_multiple(dataset, ["date_block_num", "shop_id", "item_id"], "item_cnt_day", repl="item_cnt_mnth")
#X_ = np.array(postval_set[["date_block_num", "shop_id","item_id"]])
#y_ = np.array(postval_set[["item_cnt_mnth"]])

#regr = SVR(C=10, epsilon = .1, gamma=.01, kernel="sigmoid")
regr = SVR(C=10, epsilon = .1, gamma=.01, kernel="rbf")

#regr = SVR(C=10, epsilon = .1, gamma=.01, kernel="poly", degree=15)
regr.fit(X, y.ravel())
preds = np.round(regr.predict(X_test))
print(regr.score(X_test, y_test.ravel()))
print(regr.score(X, y.ravel()))

#true accuracy function
# add up total correct, divide by total
#true = np.sum((preds == y_test.ravel() )) / len(y_test) 

print("true accuracy is ", true)



print("Hyper Parameter Tuning")
'''
#parameters = {'kernel':('sigmoid', 'poly'), 'C':[.1, 10, 1000], 'epsilon':[.001, .1], 'gamma' : [.001, 1, 1000], 'degree' : [3, 7]}
Par1 = {'kernel' : ['sigmoid', 'rbf'], 'C': [.001, .1, 100, 1000], 'epsilon':[.1], 'gamma': [.0001, .001, .01, .1]}

clf1 = GridSearchCV(regr, Par1)
clf1.fit(X, y.ravel())

print(clf1.score(X_test, y_test.ravel()))
print(clf1.best_estimator_)

preds = np.round(clf1.predict(X_test))
true = np.sum((preds == y_test.ravel() )) / len(y_test) 
print("true accuracy is ", true)

#Par2 = {'kernel' : ['poly'], 'C': [.1, 1], 'epsilon':[.1], 'gamma': [.1, 10], 'degree':[5]}

#clf2 = GridSearchCV(regr, Par2)
#clf2.fit(X, y.ravel())

#print(clf2.score(X_test, y_test.ravel()))
#print(clf2.best_estimator_)

#preds = np.round(clf2.predict(X_test))
#true = np.sum((preds == y_test.ravel() )) / len(y_test) 
#print("true accuracy is ", true)
'''

0.5734857195981953
0.6049434148817315
true accuracy is  0.9182561307901907
Hyper Parameter Tuning


'\n#parameters = {\'kernel\':(\'sigmoid\', \'poly\'), \'C\':[.1, 10, 1000], \'epsilon\':[.001, .1], \'gamma\' : [.001, 1, 1000], \'degree\' : [3, 7]}\nPar1 = {\'kernel\' : [\'sigmoid\', \'rbf\'], \'C\': [.001, .1, 100, 1000], \'epsilon\':[.1], \'gamma\': [.0001, .001, .01, .1]}\n\nclf1 = GridSearchCV(regr, Par1)\nclf1.fit(X, y.ravel())\n\nprint(clf1.score(X_test, y_test.ravel()))\nprint(clf1.best_estimator_)\n\npreds = np.round(clf1.predict(X_test))\ntrue = np.sum((preds == y_test.ravel() )) / len(y_test) \nprint("true accuracy is ", true)\n\n#Par2 = {\'kernel\' : [\'poly\'], \'C\': [.1, 1], \'epsilon\':[.1], \'gamma\': [.1, 10], \'degree\':[5]}\n\n#clf2 = GridSearchCV(regr, Par2)\n#clf2.fit(X, y.ravel())\n\n#print(clf2.score(X_test, y_test.ravel()))\n#print(clf2.best_estimator_)\n\n#preds = np.round(clf2.predict(X_test))\n#true = np.sum((preds == y_test.ravel() )) / len(y_test) \n#print("true accuracy is ", true)\n'

SVR with a sigmoid kernel is the best so far with 90% accuracy with params C = 1, gamma = .001

SVR with rbf kernel 90% accuracy with params C=1000, gamma = .0001

SVR with polynomial kernel won't run

Based on the results, SVR with a sigmoid kernel will perform the best on the dataset, but given the higher score for rbf, rbf fits the data better. both will be used in the next stages.